# User to Query

In [13]:
import pandas as pd


In [14]:
user_df = pd.read_csv('../CampsiteInfo - Sheet1.csv')
user_df.head()


,Date of Query,Name,Rec Area,Weeks in Advance,Number of Nights,Weekends Only,Email
0,11/30/2024,Chad Breece,Yosemite National Park,3,1,0,cmebreece@gmail.com
1,11/30/2024,Chad Breece,Joshua Tree National Park,3,1,0,cmebreece@gmail.com


In [15]:
uniq_users = user_df['Name'].unique()
uniq_users


array(['Chad Breece'], dtype=object)

In [21]:
rec_df = pd.read_csv('rec_area_names.csv')
rec_df.head()

,RecAreaID,RecAreaName
0,4,WestWorld
1,6,Lake Pleasant
2,9,Shadow Mountain Lake
3,10,White Mountains National Recreation Area
4,11,Folsom Lake


In [24]:
uniq_recs = []
for user in uniq_users:
    user_recs = user_df[user_df['Name'] == user]['Rec Area'].unique()
    uniq_recs.append([int(rec_df[rec_df['RecAreaName'] == rec]['RecAreaID'].values[0]) for rec in user_recs])
print(uniq_users, uniq_recs)
print(uniq_users[0], uniq_recs[0])


['Chad Breece'] [[2991, 2782]]
Chad Breece [2991, 2782]


In [29]:
from datetime import datetime
import logging
from typing import List
import pandas as pd
from datetime import datetime

from camply.containers import AvailableCampsite, SearchWindow
from camply.search import SearchRecreationDotGov

In [26]:
AreaCSV = pd.read_csv('../rec_area_names.csv')
print(AreaCSV.head())

   RecAreaID                               RecAreaName
0          4                                 WestWorld
1          6                             Lake Pleasant
2          9                      Shadow Mountain Lake
3         10  White Mountains National Recreation Area
4         11                               Folsom Lake


In [28]:
user_df = pd.read_csv('../CampsiteInfo - Sheet1.csv')
print(user_df.head())

  Date of Query         Name                   Rec Area  Weeks in Advance  \
0    11/30/2024  Chad Breece     Yosemite National Park                 3   
1    11/30/2024  Chad Breece  Joshua Tree National Park                 3   

   Number of Nights  Weekends Only                Email  
0                 1              0  cmebreece@gmail.com  
1                 1              0  cmebreece@gmail.com  


In [ ]:

uniq_users = user_df['Name'].unique()
for user in uniq_users:
    uniq_recs = user_df[user_df['Name'] == user]['Rec Area'].unique()
    
    #get number of weeks of advanced notice
    adv_notice_weeks = user_df[user_df['Name'] == user]['Weeks in Advance'].unique()
    #calculate end date
    adv_notice_days = adv_notice_weeks * 7
    u = datetime.datetime.strptime("2011-01-01","%Y-%m-%d")
    d = datetime.timedelta(days=adv_notice_days)
    start = u
    end = u + d
    date_range = SearchWindow(start_date = start, end_date = end)
    weekend_only_bool = user_df[user_df['Name'] == user]['Weekends Only'].unique()
    num_nights = user_df[user_df['Name'] == user]['Number of Nights'].unique()
    
    #Need a seperate thread for each user
    camping_finder = SearchRecreationDotGov(search_window=date_range,
                                            recreation_area=2782,  # Glacier Ntl Park
                                            weekends_only=weekend_only_bool[0],
                                            nights=num_nights[0],
                                            )
    
    matches: List[AvailableCampsite] = camping_finder.get_matching_campsites(log=True, verbose=True,
                                      continuous=True,
                                      polling_interval=5,
                                      search_forever=True,
                                      notify_first_try=False)
    
print(datetime.today())
    
print(user, uniq_recs)